In [1]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ["OPENAI_MODEL_NAME"] = (
    "gpt-4o"  # we need a larger context window for this project
)

In [2]:
from utils import get_google_api_client

google_api_client = get_google_api_client()

In [3]:
from crewai_tools import FileReadTool, SerperDevTool

transcript_path = "kt_transcripts/663.txt"

file_read_tool = FileReadTool(transcript_path)
search_tool = SerperDevTool()

In [4]:
from crewai import Agent

host = Agent(
    role="Host of Kill Tony",
    goal="Interview comedians and provide entertaining commentary",
    backstory="""
    Tony Hinchcliffe is the outspoken and controversial host of the Kill Tony podcast. 
    Tony developed his abrasive and insult-driven comedy style as a defense mechanism from an early age.
    Tony's sharp wit and willingness to push boundaries have made Kill Tony one of the most popular live comedy shows, 
    giving a platform to up-and-coming comedians while subjecting them to his signature roasting style.
    """,
    verbose=True,
    tools=[file_read_tool, search_tool],
    allow_delegation=True,
)

comedian = Agent(
    role="Comedian on Kill Tony",
    goal="Perform a 1-minute standup set and engage in an interview",
    backstory="""
    You are an aspiring comedian who has signed up for the Kill Tony podcast, a live comedy show where comedians perform a 1-minute set and are then interviewed by the host, Tony Hinchcliffe. 
    The show is known for its unpredictable and often brutal roasting of the performers, so you'll need to be prepared for anything. 
    Whether you bomb or kill, the experience of performing on Kill Tony can be a valuable opportunity to showcase your talent""",
    verbose=True,
    tools=[file_read_tool, search_tool],
    allow_delegation=True,
)

In [5]:
from crewai import Task

perform_standup_task = Task(
    description="Perform a hilarious and original 1-minute standup comedy set that showcases your unique comedic voice and perspective. You'll be taking the stage at the world-famous Kill Tony show in Austin, Texas, where hundreds of raucous comedy fans have gathered for a night of unpredictable laughs. With only 60 seconds, you'll need to captivate the audience with your sharpest material and fearless stage presence. The pressure is on to deliver a killer set that will have the crowd roaring and set you up for success in the upcoming interview portion of the show.",
    agent=comedian,
    expected_output="A standup comedy set that is 1 minute in length and showcases your unique style and voice.",
    tools=[search_tool],
    output_file="L6b_output/perform_standup_task_output.md",
)

interview_task = Task(
    description="""
    As the outspoken and controversial host of Kill Tony, you'll be interviewing the comedian immediately after their 1-minute standup set.
    Your goal is to provide entertaining commentary while also digging deeper into their background, inspirations, and the thought process behind their jokes. 
    Use your biting wit and insult-driven comedy style to roast the comedian, but also ask probing questions that surface the key traits that make them unique. 
    Don't be afraid to push boundaries and challenge them on their material, as the audience is eager to see how they handle your unfiltered critique.
    """,
    agent=host,
    expected_output="A back and forth interview with the comedian that showcases your sharp wit and ability to engage with guests in a humorous and entertaining manner.",
    output_file="L6b_output/interview_task_output.md",
    context=[perform_standup_task],
    tools=[file_read_tool],
)

In [6]:
from crewai import Crew, Process

crew = Crew(
    agents=[host, comedian],
    tasks=[perform_standup_task, interview_task],
    process=Process.sequential,
    verbose=True,
)

In [7]:
crew.kickoff()

 [DEBUG]: == Working Agent: Comedian on Kill Tony
 [INFO]: == Starting Task: Perform a hilarious and original 1-minute standup comedy set that showcases your unique comedic voice and perspective. You'll be taking the stage at the world-famous Kill Tony show in Austin, Texas, where hundreds of raucous comedy fans have gathered for a night of unpredictable laughs. With only 60 seconds, you'll need to captivate the audience with your sharpest material and fearless stage presence. The pressure is on to deliver a killer set that will have the crowd roaring and set you up for success in the upcoming interview portion of the show.


> Entering new CrewAgentExecutor chain...
To craft a hilarious and original 1-minute standup comedy set, I need to draw from various sources of inspiration and possibly some current events or relatable topics. Given the tools at my disposal, I should start by conducting some research to find current trends or funny news stories that could spark some ideas.

Action

"A back-and-forth interview between Tony and the comedian that showcases Tony's sharp wit and ability to engage with guests in a humorous and entertaining manner."

In [8]:
from IPython.display import Markdown, display

display(Markdown(perform_standup_task.output_file))

"Hey everyone, how’s it going? I just read some weird news, and I had to share it with you. Apparently, in California, animal rescuers and utility workers teamed up to save a cat that was stuck on top of a utility pole for several days. I mean, this cat had a better view than most people in San Francisco! It’s like the feline version of a vacation in the Alps.

Speaking of weird rescues, did you hear about the guy who got arrested for trying to rob a bank with a live beehive? I guess he thought he could make a 'buzzing' getaway. The police were like, 'Sir, we’re going to need you to BEE-have.'

And in other news, scientists discovered ancient predatory worms that are making them rethink the history of life on Earth. Predatory worms! Just when you thought your nightmares couldn't get worse. Forget sharks; now we have to worry about killer spaghetti.

Oh, and Christmas is coming early this year, and by early, I mean it’s already in stores. I walked into a shop, and I got hit by a wave of Christmas decorations. It’s like Halloween didn’t even get a chance to say 'Boo!' before Santa came in with his ho-ho-ho.

Thanks, everyone, that’s my time!"

In [9]:
display(Markdown(interview_task.output_file))

A back-and-forth interview between Tony and the comedian that showcases Tony's sharp wit and ability to engage with guests in a humorous and entertaining manner.